<a href="https://colab.research.google.com/github/bobbercheng/Advanced-Machine-Learning-Specialization/blob/master/feedback_prize_2021_blurr_bigbird_roberta_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
  from google.colab import drive
  drive.mount("/gdrive", force_remount=True)


  !pip uninstall -y kaggle
  !pip install kaggle
  !mkdir /root/.kaggle
  !cp /gdrive/MyDrive/kaggle/kaggle.json /root/.kaggle

  !kaggle competitions download -c  feedback-prize-2021
  !mkdir -p ../input/feedback-prize-2021
  !unzip -n -qq 'feedback-prize-2021.zip' -d ../input/feedback-prize-2021

  !kaggle datasets download -d  cdeotte/py-bigbird-v26
  !mkdir -p ../input/py-bigbird-v26
  !unzip -n -qq py-bigbird-v26.zip -d ../input/py-bigbird-v26

  !ln -s /gdrive/MyDrive/kaggle/feedback-prize-2021/blurr-models/bigbird-roberta-base models

  !pip install -Uqq fastcore --upgrade
  !pip install -Uqq fastai --upgrade
  !pip install transformers


Mounted at /gdrive
Found existing installation: kaggle 1.5.12
Uninstalling kaggle-1.5.12:
  Successfully uninstalled kaggle-1.5.12
  Using cached kaggle-1.5.12-py3-none-any.whl
mkdir: cannot create directory ‘/root/.kaggle’: File exists
feedback-prize-2021.zip: Skipping, found more recently modified local copy (use --force to force download)
py-bigbird-v26.zip: Skipping, found more recently modified local copy (use --force to force download)
ln: failed to create symbolic link 'models/bigbird-roberta-base': File exists


In [2]:
%%bash
git clone https://github.com/ohmeow/blurr.git
cd blurr
pip install -e ".[dev]"

Obtaining file:///content/blurr
  Attempting uninstall: ohmeow-blurr
    Found existing installation: ohmeow-blurr 0.1.2
    Can't uninstall 'ohmeow-blurr'. No files were found to uninstall.
  Running setup.py develop for ohmeow-blurr


fatal: destination path 'blurr' already exists and is not an empty directory.


In [3]:
!pip show ohmeow-blurr

Name: ohmeow-blurr
Version: 0.1.2
Summary: A library that integrates huggingface transformers with version 2 of the fastai framework
Home-page: https://github.com/ohmeow/blurr/tree/master/
Author: Wayde Gilliam
Author-email: wgilliam@ohmeow.com
License: Apache Software License 2.0
Location: /content/blurr
Requires: torch, fastai, transformers, datasets, sentencepiece, seqeval, rouge-score, nbdev
Required-by: 


In [4]:
import pandas as pd
import tqdm
import os

import sys
sys.path.append('/content/blurr')

import torch
from transformers import *
from fastai.text.all import *

from blurr.data.all import *
from blurr.modeling.all import *




In [5]:
train_df = pd.read_csv('../input/feedback-prize-2021/train.csv')

In [6]:
print( train_df.shape )
train_df.head(20)

(144293, 8)


,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring
0,423A1CA112E2,1.622628e+12,8.0,229.0,Modern humans today are always on their phone. They are always on their phone more than 5 hours a day no stop .All they do is text back and forward and just have group Chats on social media. They even do it while driving.,Lead,Lead 1,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44
1,423A1CA112E2,1.622628e+12,230.0,312.0,They are some really bad consequences when stuff happens when it comes to a phone.,Position,Position 1,45 46 47 48 49 50 51 52 53 54 55 56 57 58 59
2,423A1CA112E2,1.622628e+12,313.0,401.0,Some certain areas in the United States ban phones from class rooms just because of it.,Evidence,Evidence 1,60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75
3,423A1CA112E2,1.622628e+12,402.0,758.0,"When people have phones, they know about certain apps that they have .Apps like Facebook Twitter Instagram and Snapchat. So like if a friend moves away and you want to be in contact you can still be in contact by posting videos or text messages. People always have different ways how to communicate with a phone. Phones have changed due to our generation.",Evidence,Evidence 2,76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138
4,423A1CA112E2,1.622628e+12,759.0,886.0,Driving is one of the way how to get around. People always be on their phones while doing it. Which can cause serious Problems.,Claim,Claim 1,139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
5,423A1CA112E2,1.622628e+12,887.0,1150.0,That's why there's a thing that's called no texting while driving. That's a really important thing to remember. Some people still do it because they think It's stupid. No matter what they do they still have to obey it because that's the only way how did he save.,Evidence,Evidence 3,163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210
6,423A1CA112E2,1.622628e+12,1151.0,1533.0,"Sometimes on the news there is either an accident or a suicide. It might involve someone not looking where they're going or tweet that someone sent. It either injury or death. If a mysterious number says I'm going to kill you and they know where you live but you don't know the person's contact\n\n,It makes you puzzled and make you start to freak out. Which can end up really badly.",Evidence,Evidence 4,211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 278 279 280 281
7,423A1CA112E2,1.622628e+12,1534.0,1602.0,Phones are fine to use and it's also the best way to come over help.,Claim,Claim 2,282 283 284 285 286 287 288 289 290 291 292 293 294 295 296
8,423A1CA112E2,1.622628e+12,1603.0,1890.0,"If you go through a problem and you can't find help you ,always have a phone there with you. Even though phones are used almost every day as long as you're safe it would come into use if you get into trouble. Make sure you do not be like this phone while you're in the middle of driving.",Evidence,Evidence 5,297 298 299 300 301 302 303 304 305 306 307 308 309 310 311 312 313 314 315 316 317 318 319 320 321 322 323 324 325 326 327 328 329 330 331 332 333 334 335 336 337 338 339 340 341 342 343 344 345 346 347 348 349 350 351 352 353 354
9,423A1CA112E2,1.622628e+12,1891.0,2027.0,The news always updated when people do something stupid around that involves their phones. The safest way is the best way to 

In [7]:
# https://www.kaggle.com/raghavendrakotala/fine-tunned-on-roberta-base-as-ner-problem-0-533
test_names, test_texts = [], []
for f in list(os.listdir('../input/feedback-prize-2021/test')):
    test_names.append(f.replace('.txt', ''))
    test_texts.append(open('../input/feedback-prize-2021/test/' + f, 'r').read())
test_texts = pd.DataFrame({'id': test_names, 'text': test_texts})
test_texts.head()

,id,text
0,18409261F5C2,"80% of Americans believe seeking multiple opinions can help them make better choices, and for good reason. Studies have shown the average Americans faring far better in their lives compared to their counterparts because they are listening to other's advice. There are also many myths that have the moral of listening to other people's opinions. For example, Perseus got his achievement of slaying a gorgon because he listened to the Oracle. Another example I have is the fable of Osiris, in which Osiris listens to Sekhmet and becomes the king of he underworld. In all of these stories, the hero ..."
1,0FB0700DAF44,"During a group project, have you ever asked a group member about adding or replacing something? Or, when you were studying for a math test, did you ever ask your parents or sibling about different ways to tackle a certain problem? Asking for other's opinions is especially beneficial as it allows for an individual to receive a variety of different views towards a given topic. Likewise, being diverse and asking many people for their opinions allows one to understand how most people percieve something. This is especially important as knowing multiple opinions can allow someone to take those v..."
2,D72CB1C11673,"Making choices in life can be very difficult. People often ask for advice when they can not decide on one thing. It's always good to ask others for their advice when making a choice. When you have multiple opinions you have the ability to make the best choice for yourself. Seeking multiple opinions can help a person make a better choice because it can decrease stress levels, a great chance to learn something new, can be very helpful and beneficial.\n\nTo begin with, Seeking information from more than one person can decrease stress levels. When making a decision there is a chance that you c..."
3,D46BCB48440A,"When people ask for advice,they sometimes talk to more than one person. Advice is an opinion,that's told to another person. I think advises help people make better choices. Some reasons I think why advises helps is keeps you safe, makes goal come true, and makes changes.\n\nThe reason I think advises help is keeps you safe. There are many dangerous people and thing in the world. That could hurt you in so many ways. When I got invited to a sixteen and up birthday party. I was so ready to go but i didnt know if i should go to the party. So i asked my mom for advice. Then she told me it would..."
4,DF920E0A7337,"Have you ever asked more than one person for help on what product to buy in a situation? Interviews are a perfect example of seeking more than one opinion. Interviews are seen on the news, in professional sports, and other places as well. There are a lot of people that can help you buy the right product as well. They can all open you up to new ideas. I think talking to more than one person on an opinion is a better choice because it shows you more than one option, it can change your perspective of a topic, and it informs you about what other people enjoy.\n\nFirst, I think it shows you mor..."


In [8]:
# https://www.kaggle.com/raghavendrakotala/fine-tunned-on-roberta-base-as-ner-problem-0-533
test_names, train_texts = [], []
for f in list(os.listdir('../input/feedback-prize-2021/train')):
    test_names.append(f.replace('.txt', ''))
    train_texts.append(open('../input/feedback-prize-2021/train/' + f, 'r').read())
train_text_df = pd.DataFrame({'id': test_names, 'text': train_texts})
train_text_df.head()

,id,text
0,1D7EEE65F780,"Wouldn't it be great to be able to take some of your high school courses in the the comfort of your own home? Some schools already offer classes online or let students take them through video conferencing from their homes. Although some may say otherwise, Economics and Personal Finance, Testout for Computer Systems Tech, and online college courses prove that students would benefit from being able to attend classes from home.\n\nThere is an option for students to take EPF over the summer at home as opposed to during the school year, which is very helpful to students who want to get one of t..."
1,201EB1D0F1D5,"Dear Principal,\n\nI would possibly prefer Policy 2 because if you would let students use their cellular devices or just cell phones they would think that if you would let them use them during their free time they would use it during the class time too. The students really don't care what class or free time the get to do it depends on how they want to do it. Students would rather want to get in trouble than following the rules at school.\n\nStudents also think its okay to text during class as long as the teacher doesn't see it or if another student sees it but does not tell on the person w..."
2,8BBDEC4FD82E,"It said the cowboys played baseball,volleyball,table tennis tornaments,fencing,boxing,reading,whittling,and games that help pass time so i think he was like showing people and telling people about its so fun and it was like about world war 2 and like that and told him it would be awesome and fun and all like that and he was telling a hole bunch of people to join and that it would be really fun and it was gponna be about the old days and like thatand we will be doing some cool stuff about it and fun building things and like that and that your going to learn some interesting stuff and maybe ..."
3,C349F414015C,"Phone and driving\n\nThis paper explores if we need to using the cell phones while driving . The cell phone use driving while driving is one of those hot -button issue at the moment .Everyone has a cell phone,when it goes off while they are driving whether it is a message,social media or if someone call you .You need to see what happens to your phone when someone text to you. Using the cell phone while driving should be banned completely for the safety of everyone on the road .Here are just a few reason why the cell phone while driving should be banned.\n\nLet's start with text and social ..."
4,9078542008BD,"Dear state senator,\n\nI think we should'nt have the Electoral College because the system will go all wrong because who were actually voting for, unfair to voters, and its outdated and irrational. These are the reasons why i think that we shouldnt have Electoral College.\n\nWe shouldnt have it because voters dont actually vote for the president they want, but for a slate of eletors who is turn to elect the president which is unfair. If you wanted to vote for a president but you arent in that area you wouldnt be voting for that peron you'll be voting for a slate of 34 electors who pledge to..."


In [9]:
LOAD_TOKENS_FROM = '/gdrive/MyDrive/kaggle/feedback-prize-2021'
if not LOAD_TOKENS_FROM:
    all_entities = []
    all_tokens = []
    for ii,i in enumerate(train_text_df.iterrows()):
        if ii%100==0: print(ii,', ',end='')
        tokens = i[1]['text'].split()
        all_tokens.append(tokens)
        total = tokens.__len__()
        entities = ["O"]*total
        for j in train_df[train_df['id'] == i[1]['id']].iterrows():
            discourse = j[1]['discourse_type']
            list_ix = [int(x) for x in j[1]['predictionstring'].split(' ')]
            entities[list_ix[0]] = f"B-{discourse}"
            for k in list_ix[1:]: entities[k] = f"I-{discourse}"
        all_entities.append(entities)
    train_text_df['entities'] = all_entities
    train_text_df['tokens'] = all_tokens
    train_text_df.to_csv(f'{LOAD_TOKENS_FROM}/train_NER.csv',index=False)
    
else:
    from ast import literal_eval
    train_text_df = pd.read_csv(f'{LOAD_TOKENS_FROM}/train_NER.csv')
    # pandas saves lists as string, we must convert back
    train_text_df.entities = train_text_df.entities.apply(lambda x: literal_eval(x) )
    train_text_df.tokens = train_text_df.tokens.apply(lambda x: literal_eval(x) )
    
print( train_text_df.shape )
train_text_df.head()

(15594, 4)


,id,text,entities,tokens
0,52233E7A7574,"Helping the community is a good thing, but people don't have to do it if they don't want to. But students should help out the community. i think since that we live in a good/bad community i think it would be good to help out the environment, or help younger individuals get a better education. Other community services require like babysitting, donating food, or donate money, not all community services are long and time wasting.\n\nIf all the students did community service, then the community would be way better for the young and the old. examples of a bad community for the young and the old...","[B-Lead, I-Lead, I-Lead, I-Lead, I-Lead, I-Lead, I-Lead, I-Lead, I-Lead, I-Lead, I-Lead, I-Lead, I-Lead, I-Lead, I-Lead, I-Lead, I-Lead, I-Lead, I-Lead, B-Position, I-Position, I-Position, I-Position, I-Position, I-Position, I-Position, B-Evidence, I-Evidence, I-Evidence, I-Evidence, I-Evidence, I-Evidence, I-Evidence, I-Evidence, I-Evidence, I-Evidence, I-Evidence, I-Evidence, I-Evidence, I-Evidence, I-Evidence, I-Evidence, I-Evidence, I-Evidence, I-Evidence, I-Evidence, I-Evidence, I-Evidence, I-Evidence, I-Evidence, I-Evidence, I-Evidence, I-Evidence, I-Evidence, I-Evidence, I-Evidence,...","[Helping, the, community, is, a, good, thing,, but, people, don't, have, to, do, it, if, they, don't, want, to., But, students, should, help, out, the, community., i, think, since, that, we, live, in, a, good/bad, community, i, think, it, would, be, good, to, help, out, the, environment,, or, help, younger, individuals, get, a, better, education., Other, community, services, require, like, babysitting,, donating, food,, or, donate, money,, not, all, community, services, are, long, and, time, wasting., If, all, the, students, did, community, service,, then, the, community, would, be, way, b..."
1,4409FED98F5A,I think that students ought to have there vote in the situation because why would you make people do something they don't want to do. I also think that they should help in some way because the more people that help the more productive and fun it will be. You can also split the people that are helping into different groups to cover places more in the region. The more we help at a young age the more we will do it through out our lives. Helping the community is the most important thing next to a education and we should all want to be a part of it no matter how young or old you are. The more t...,"[B-Position, I-Position, I-Position, I-Position, I-Position, I-Position, I-Position, I-Position, I-Position, I-Position, I-Position, I-Position, I-Position, I-Position, I-Position, I-Position, I-Position, I-Position, I-Position, I-Position, I-Position, I-Position, I-Position, I-Position, I-Position, B-Claim, I-Claim, I-Claim, I-Claim, I-Claim, I-Claim, I-Claim, I-Claim, I-Claim, I-Claim, I-Claim, I-Claim, I-Claim, I-Claim, I-Claim, I-Claim, I-Claim, I-Claim, I-Claim, I-Claim, I-Claim, I-Claim, I-Claim, I-Claim, B-Evidence, I-Evidence, I-Evidence, I-Evidence, I-Evidence, I-Evidence, I-Evide...","[I, think, that, students, ought, to, have, there, vote, in, the, situation, because, why, would, you, make, people, do, something, they, don't, want, to, do., I, also, think, that, they, should, help, in, some, way, because, the, more, people, that, help, the, more, productive, and, fun, it, will, be., You, can, also, split, the, people, that, are, helping, into, different, groups, to, cover, places, more, in, the, region., The, more, we, help, at, a, young, age, the, more, we, will, do, it, through, out, our, lives., Helping, the, community, is, the, most, important, thing, next, to, a, ..."
2,2F2B6C492E97,"Dear Florida senator,\n\nI think that we should keep the Electoral College vote system, because it is pretty much the same thing as election by popular vote. When you vote for your canidatate, you're voting for his electors. Then those electors vote for their presidential nominee. Not often do the electors betray thei

In [10]:
!cat ../input/feedback-prize-2021/train/52233E7A7574.txt

Helping the community is a good thing, but people don't have to do it if they don't want to. But students should help out the community. i think since that we live in a good/bad community i think it would be good to help out the environment, or help younger individuals get a better education. Other community services require like babysitting, donating food, or donate money, not all community services are long and time wasting.

If all the students did community service, then the community would be way better for the young and the old. examples of a bad community for the young and the old would be like pollution in the air and on the ground, if everyone was smoking it would increase the chances of the elderly and young to get a lung disease like asthma. Or if we left litter on the floor babies could eat the litter and could go to the hospital or even die.

i recommend that everyone does community service even if the students don't want to, its good for the environment, the people, and t

In [11]:
# CREATE DICTIONARIES THAT WE CAN USE DURING TRAIN AND INFER
output_labels = ['O', 'B-Lead', 'I-Lead', 'B-Position', 'I-Position', 'B-Claim', 'I-Claim', 'B-Counterclaim', 'I-Counterclaim', 
          'B-Rebuttal', 'I-Rebuttal', 'B-Evidence', 'I-Evidence', 'B-Concluding Statement', 'I-Concluding Statement']

labels_to_ids = {v:k for k,v in enumerate(output_labels)}
ids_to_labels = {k:v for k,v in enumerate(output_labels)}

In [12]:
labels_to_ids

{'B-Claim': 5,
 'B-Concluding Statement': 13,
 'B-Counterclaim': 7,
 'B-Evidence': 11,
 'B-Lead': 1,
 'B-Position': 3,
 'B-Rebuttal': 9,
 'I-Claim': 6,
 'I-Concluding Statement': 14,
 'I-Counterclaim': 8,
 'I-Evidence': 12,
 'I-Lead': 2,
 'I-Position': 4,
 'I-Rebuttal': 10,
 'O': 0}

In [13]:
# Refer to https://github.com/ohmeow/blurr/blob/master/nbs/03_modeling-token-classification.ipynb

## High-level API BLearnerForTokenClassification

In [14]:
import torch
import gc

torch.cuda.empty_cache()
gc.collect()

347

In [15]:
pretrained_model_name = 'google/bigbird-roberta-base'

learn = BlearnerForTokenClassification.from_dataframe(train_text_df, pretrained_model_name, 
                                                      tokens_attr='tokens', token_labels_attr='entities', 
                                                      dblock_splitter=RandomSplitter(), 
                                                      dl_kwargs={'bs':2}).to_fp16()

learn.unfreeze()

In [16]:
learn.dls.show_batch(dataloaders=learn.dls, max_n=2)

In [17]:
# learn.lr_find()

In [18]:
%%time
#17 minutes
learn.fit_one_cycle(10, lr_max=1e-4, moms=(0.8,0.7,0.8), cbs=[BlearnerForTokenClassification.get_metrics_cb(),
                                                             SaveModelCallback(),
                                                             EarlyStoppingCallback(monitor='f1', comp=np.greater, patience=2)])

epoch,train_loss,valid_loss,accuracy,precision,recall,f1,time


RuntimeError: ignored

In [ ]:
learn.export('models/longformer-base-4096-ner.pkl')

In [ ]:
learn.show_results(learner=learn, max_n=2, trunc_at=10)

In [ ]:
#slow
print(learn.token_classification_report)